# Convolutional Neural Network (CNN) For Detecting Defective Parts

This notebook uses the [`TensorFlow`](https://www.tensorflow.org/) library to implement a Convolutional Neural Network (CNN) classifier with various hyperparameters. The CNN classifier is used to detect defective mechanical parts in a manufacturing process. The image data we used is from the ["Casting Product Image Data For Quality Inspection"](https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product) dataset, and contains 8646 images used for quality inspection that we used to train the CNN.

First, import the necessary libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import json
from tqdm.auto import tqdm

Declare global variables

In [6]:

DATA_PATH = './casting_512x512/casting_512x512/'
OUTPUT_PATH = './out/'
TRAIN_TEST_SPLIT = 0.8
IMG_PX_SIZE = 128
RESIZE_SHAPE = (IMG_PX_SIZE, IMG_PX_SIZE)
INPUT_SHAPE = (IMG_PX_SIZE, IMG_PX_SIZE, 3)

Methods for loading image data, flattening the images, shuffling them, and splitting them into training and test sets. (Credit to Sam Little)

Because CNN's and neural networks in general benefit from larger datasets than SVM's, each image is duplicated and flipped about the vertical axis to double the size of training and testing sets.

In [4]:

def load_image_data():
  data = []
  labels = []
  print('Loading OK images...')
  for filename in tqdm(os.listdir(f'{DATA_PATH}/ok_front')):
    img = Image.open(f'{DATA_PATH}/ok_front/{filename}')
    img_f = img.transpose(Image.FLIP_LEFT_RIGHT)
    data.append(img)
    labels.append(0)
    data.append(img_f)
    labels.append(0)
  print('Loading defective images...')
  for filename in tqdm(os.listdir(f'{DATA_PATH}/def_front')):
    img = Image.open(f'{DATA_PATH}/def_front/{filename}')
    img_f = img.transpose(Image.FLIP_LEFT_RIGHT)
    data.append(img)
    labels.append(1)
    data.append(img_f)
    labels.append(1)
  return data, labels

def resize_images(data, shape):
  resized_data = []
  print(f'Resizing images to {shape}...')
  for img in tqdm(data):
    resized_data.append(img.resize(shape))
  return resized_data

def prepare_image_data(data, labels):
  images = []
  print('Preparing images...')
  for img in tqdm(data):
    img = np.array(img)
    img = img / 255.0
    img[img == 0] = 0.0001
    images.append(img)
  data = np.array(images)
  original_shape = data[0].shape
  #data = data.reshape(data.shape[0], -1)
  shuffled = list(zip(data, labels))

  np.random.shuffle(shuffled)
  train = shuffled[:int(len(shuffled) * TRAIN_TEST_SPLIT)]
  test = shuffled[int(len(shuffled) * TRAIN_TEST_SPLIT):]

  train_data = np.array([i[0] for i in train])
  train_labels = np.array([i[1] for i in train])
  test_data = np.array([i[0] for i in test])
  test_labels = np.array([i[1] for i in test])

  return train_data, train_labels, test_data, test_labels, original_shape

Declare hyperparameters we wish to test, and feed each into an `TensorFlow` CNN, fitting it to the training data. Accuracy, confusion matrices, and other metrics are generated for each combination.

All images are initally resized to 128x128 for the sake of computational speed, and the two convolutional layers further reduce them to 32x32, before being fed through the two densly connected layers

In [ ]:

data, labels = load_image_data()
data = resize_images(data, RESIZE_SHAPE)
train_data, train_labels, test_data, test_labels, original_shape = prepare_image_data(data, labels)

optimizers_to_test = ['rmsprop', 'sgd', 'adam', 'adadelta']
activations_to_test = ['relu', 'sigmoid']
loss_func_to_test = ['sparse_categorical_crossentropy', 'mse', 'categorical_hinge']
layer1_sizes_to_test = [128, 256, 512]
layer2_sizes_to_test = [16, 32, 64]

tests_to_run = []

for loss in loss_func_to_test:
  for optimizer in optimizers_to_test:
    for activation in activations_to_test:
      for layer1 in layer1_sizes_to_test:
        for layer2 in layer2_sizes_to_test:
          tests_to_run.append({
            'name': f'{loss}_{optimizer}_{activation}_{layer1}_{layer2}',
            'loss': loss,
            'optimizer': optimizer,
            'activation': activation,
            'layer1': layer1,
            'layer2': layer2,
          })
          
results = [] 
           
for idx, test in enumerate(tests_to_run):
  
  print('\n' + 'Test ', idx+1, '/', len(tests_to_run), ' - ', test['name'])
  
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(test['layer1'], activation=test['activation']),
    tf.keras.layers.Dense(test['layer2'], activation=test['activation']),
    tf.keras.layers.Dense(2, activation='softmax')
])

  model.compile(optimizer=test['optimizer'],
                loss=test['loss'],
                metrics=['accuracy'])

  model.fit(train_data, train_labels, epochs=10)

  test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

  predictions = np.argmax(model.predict(test_data), axis=1)
  accuracy = accuracy_score(test_labels, predictions)
  conf_matrix = confusion_matrix(test_labels, predictions)
  report = classification_report(test_labels, predictions)
  results.append({
    'name': test['name'],
    'loss': test['loss'],
    'optimizer': test['optimizer'],
    'activation': test['activation'],
    'layer1': test['layer1'],
    'layer2': test['layer2'],
    'accuracy': accuracy,
    'confusion_matrix': conf_matrix.tolist(),
    'classification_report': report,
  })

For each parameter combination, we save the confusion matrix, accuracy, and other metrics to the output directory. (Credit to Sam Little)

In [ ]:

for result in results:
  result_path = f'{OUTPUT_PATH}/{result["loss"]}/{result["optimizer"]}/{result["activation"]}'
  # if path doesn't exist, create it
  if not os.path.exists(result_path):
    os.makedirs(result_path)

  # save confusion matrix
  plt.figure(figsize=(10, 10))
  plt.imshow(result['confusion_matrix'], interpolation='nearest', cmap=plt.cm.Blues)
  plt.title(f'Confusion Matrix for {result["name"]}\nAccuracy: {result["accuracy"] * 100:.2f}%\nImage Size: {RESIZE_SHAPE}')
  plt.colorbar()
  tick_marks = np.arange(2)
  plt.xticks(tick_marks, ['OK', 'DEFECTIVE'], rotation=45)
  plt.yticks(tick_marks, ['OK', 'DEFECTIVE'])
  plt.tight_layout()
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label')
  plt.savefig(f'{result_path}/confusion_matrix.png')

  # save results to JSON
  with open(f'{result_path}/{result["name"]}.json', 'w') as f:
    result = {
      'name': result['name'],
      'loss': result['loss'],
      'optimizer': result['optimizer'],
      'activation': result['activation'],
      'layer1': test['layer1'],
      'layer2': test['layer2'],
      'accuracy': result['accuracy'],
      'confusion_matrix': result['confusion_matrix'],
      'confusion_matrix_image': f'{result_path}/confusion_matrix.png',
      'classification_report': result['classification_report']
    }
    json.dump(result, f)
    
    
results.sort(key=lambda x: x['accuracy'], reverse=True)
print("Best results:")
for index, result in enumerate(results):
  print(f'{index + 1}. {result["name"]} - {result["accuracy"] * 100:.2f}%')